**快速幂**（Exponentiation by squaring，平方求幂）是一种简单而有效的小算法，它可以以$O(logn)$的时间复杂度计算乘方。快速幂不仅本身非常常见，而且后续很多算法也都会用到快速幂。<br/>

---
让我们先来思考一个问题：**7的10次方，怎样算比较快？**

- **方法1：**最朴素的想法，7 * 7=49，49 * 7=343，... 一步一步算，共进行了9次乘法。

这样算无疑太慢了，尤其对计算机的CPU而言，每次运算只乘上一个个位数，无疑太屈才了。这时我们想到，也许可以拆分问题。

- **方法2：**先算7的5次方，即7 * 7 * 7 * 7 * 7，再算它的平方，共进行了5次乘法。

但这并不是最优解，因为对于“7的5次方”，我们仍然可以拆分问题。

- **方法3：**先算7*7得49，则7的5次方为49 * 49 * 7，再算它的平方，共进行了4次乘法。

模仿这样的过程，我们得到一个在$O(logn)$时间内计算出幂的算法，也就是快速幂。

---
### 递归快速幂

刚刚我们用到的，无非是一个二分的思路。我们很自然地可以得到一个递归方程：

!["快速幂递归"](./ksmdgequation.svg)
计算a的n次方，如果n是偶数（不为0），那么就先计算a的n/2次方，然后平方；如果n是奇数，那么就先计算a的n-1次方，再乘上a；递归出口是a的0次方为1。

递归快速幂的思路非常自然，代码也很简单（直接把递归方程翻译成代码即可）：

```
def fastmi(i,n):
    if n==0:
        return 1
    elif n % 2 == 1:
        return fastmi(i,n-1)*a
    else:
        t=fastmi(i,n//2)
        return t*t
```

---
### 非递归快速幂

我们换一个角度来引入非递归的快速幂。还是7的10次方，但这次，我们把10写成二进制的形式，也就是 $(1010)_2$ 。

现在我们要计算$7^{(1010)_2}$ ，可以怎么做？我们很自然地想到可以把它拆分为$7^{(1000)_2} 7^{(10)_2}$ . 实际上，对于任意的整数，我们都可以把它拆成若干个 $7^{(1000...)_2}$ 的形式相乘。而这些$7^{(1000...)_2}$，恰好就是 $7^1$、$7^2$、$7^4$……我们只需不断把底数平方就可以算出它们。

我们先看代码，再来仔细推敲这个过程：
```
def fastmi(i,n):
    ans = 1
    while n > 0:
        if n&1==1:
            ans*=i
        i*=i
        n=n>>1
    return ans
```
最初ans为1，然后我们一位一位算：

1010的最后一位是0，所以a^1这一位不要。然后1010变为101，a变为a^2。

101的最后一位是1，所以a^2这一位是需要的，乘入ans。101变为10，a再自乘。

10的最后一位是0，跳过，右移，自乘。

然后1的最后一位是1，ans再乘上a^8。循环结束，返回结果。

非递归版比递归版的速度慢

In [69]:
def fastmi(i,n):
    if n==0:
        return 1
    elif n % 2 == 1:
        return fastmi(i,n-1)*i
    else:
        t=fastmi(i,n//2)
        return t*t
def fastmi_nor(i,n):
    ans = 1

    while n > 0:
        if n&1==1:
            ans=ans*i
        i = i*i
        n = n>>1
    return ans
def normi(i,n):
    ans = 1
    for _ in range(n):
        ans*=i
    return ans

%time fastmi_nor(7,99999)
%time fastmi(7,99999)
%time normi(7,99999)

CPU times: user 10.5 ms, sys: 1 µs, total: 10.5 ms
Wall time: 10.3 ms
CPU times: user 3.82 ms, sys: 0 ns, total: 3.82 ms
Wall time: 3.82 ms
CPU times: user 339 ms, sys: 0 ns, total: 339 ms
Wall time: 338 ms


...